## News and stock markets

An experiment to teach a classificator to predict next day change (decrease=0, increase=1) of DJIA using pre-trained GoogleNews [word2vec model](https://github.com/mmihaltz/word2vec-GoogleNews-vectors) and bag-of-words approach on a news corpus from [kaggle](https://www.kaggle.com/aaron7sun/stocknews)

### 1. Load the data

In [79]:
import gensim
import numpy as np
import pandas as pd

In [80]:
import pandas as pd

data = pd.read_csv( "Combined_News_DJIA.csv", header=0, index_col=["Date"], parse_dates=[0])
#news corpus from kaggle
data

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',"b""The commander of a Navy air reconnaissance s...",...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'S

In [81]:
#DJIA Close values for the same period
data1 = pd.read_csv('xom.csv', header=0, sep=',', usecols=[0,5], parse_dates=[0])
data1.head(10)

,<DATE>,<CLOSE>
0,2008-08-08,78.74
1,2008-08-09,78.72
2,2008-08-11,78.17
3,2008-08-12,76.90
4,2008-08-13,77.89
5,2008-08-14,77.50
6,2008-08-15,77.06
7,2008-08-16,77.07
8,2008-08-18,76.58
9,2008-08-19,77.96


In [82]:
#mark if the DJIA goes down(0) or up (1) on the next day, will be used as labels for classification
data1['label'] = data1['<CLOSE>'].shift(-1) - data1['<CLOSE>']
for i in range(len(data1)):
    if data1.loc[i,'label'] > 0:
        data1.loc[i,'label'] = 1
    else:
        data1.loc[i,'label'] = 0
        
data1 = data1.set_index('<DATE>', drop=True)

In [83]:
data1

,<CLOSE>,label
<DATE>,,
2008-08-08,78.740,0.0
2008-08-09,78.720,0.0
2008-08-11,78.170,0.0
2008-08-12,76.900,1.0
2008-08-13,77.890,0.0
2008-08-14,77.500,0.0
2008-08-15,77.060,1.0
2008-08-16,77.070,0.0
2008-08-18,76.580,1.0


In [84]:
#combine news and labels
a = pd.merge(data, data1, how='inner', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True,
          copy=False, indicator=False)

In [85]:
a = a.reset_index()

In [86]:
a = a.dropna()
a

,index,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,<CLOSE>,label
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",78.740,0.0
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",78.170,0.0
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",76.900,1.0
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,77.890,0.0
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'War in the Caucasus is as much the product o...,"b'""Non-media"" photos of South Ossetia/Georgia ...",b'Georgian TV reporter shot by Russian sniper ...,b'Saudi Arabia: Mother moves to block child ma...,b'Taliban wages war on humani

In [136]:
import nltk
from bs4 import BeautifulSoup
import re

from nltk.corpus import stopwords

In [88]:
def to_wordlist(news):
    '''takes a sentence in, removes non-english-letter symbols and
    stop words from nltk database and outputs a list of words of the sentence'''

    # 1. Remove HTML
   # text = BeautifulSoup(news, "lxml").get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ", news)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    stops = set(stopwords.words('english'))
    words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [91]:
def get_sentences(data):
    '''processes a df with news headers using the function from above'''
    sentences = []
    for i in range(len(data)):
        if i%100 == 0:
            print(i)
        for j in range(2,27):
            
            message = data.iloc[i,j]
            
            if message is not np.nan:
                sentences.append(to_wordlist(message))
    sentences = np.asarray(sentences)
    return sentences

In [92]:
#getting an array of processed headers
sentences = get_sentences(a)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [93]:
sentences.shape

(49250,)

### 2. Loading google model and defining functions to convert news to vectors with its help

In [3]:
#edit log formatting
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)


from gensim.models import Word2Vec

In [ ]:
#load a pre-trained model from GoogleNews
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def makeFeatureVec(words, model, num_features):
    ''' Function to average all of the word vectors in a header'''
    
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(news, model=model, num_features=300):
    
    result = makeFeatureVec(news, model, num_features)

    return result

ran these functions on 47000 news in the 'sentences' list, resulting in 47000 300-dimensional vectors, which were saved in 2 files in order to not to loose them accidentally. 

load the 2 files and merge

In [21]:
import pickle
vectors = pickle.load(open('vectors', 'rb'))
vectors = vectors[:22660,:]
vectors2 = pickle.load(open('vectors2', 'rb'))
vectors2 = vectors2[22660:47000,:]
vectors = np.concatenate((vectors, vectors2))

create an array of labels, each news from the same day (25 in total) gets the same label 

In [ ]:
labels = a['label'].values
labels = np.asarray([[i]*25 for i in labels]).flatten()


#creating a dataframe for the bag-of-words approach
data_1 = pd.DataFrame({'label' : labels,
                        'news' : sentences})

###  3. Training xgboostclassifier on vectors from word2vec

without parameter optimization, just some adjustments of learning rate and tree depth performed

In [25]:
from xgboost import XGBClassifier

In [21]:
from sklearn.model_selection import train_test_split 

In [97]:
seed = 7
forest = XGBClassifier(learning_rate=0.001,max_depth=3, n_estimators=100, silent=False, objective='binary:logistic', seed=seed)
train_x,test_x, train_y, test_y = train_test_split(vectors, data_1.loc[:46999, 'label'].values,
                                                   test_size=0.33, random_state=seed)
forest.fit(train_x, train_y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.001, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=7, silent=False, subsample=1)

In [98]:
forest.score(train_x, train_y)

0.5696729120355668

In [99]:
forest.score(test_x, test_y)

0.5631850419084462

Pretty bad results, close to the chance level

### 4. Training random forest on the same word2vec data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
train_x,test_x, train_y, test_y = train_test_split(vectors, data_1.loc[:46999, 'label'].values,
                                                   test_size=0.33, random_state=seed)
rf = RandomForestClassifier(n_estimators=100, max_depth=5, criterion='entropy', random_state=seed,
                           n_jobs=-1, verbose=1)
rf.fit(train_x, train_y)
rf.score(train_x, train_y)

In [167]:
rf.score(test_x, test_y)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished


0.51308833010960675

Same result, very close to the chance level:

In [169]:
1- len(test_y[test_y==1])/len(test_y)

0.514635718891038

Conclussion: algorithms unable to classify the preprocessed news correctly according to their expected influence on tomorrows DJIA

### 5. Training xgboost on bag-of-words vectors

In [107]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [129]:
#creating a vocabulary of 2-word n-grams and vectorizing the data according to it
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             ngram_range = (2,2),
                             max_features = 40) 

vectorizer.fit(data_1['news'])

#split into train and test sets
train_x,test_x, train_y, test_y = train_test_split(vectorizer.transform(data_1['news']),
                                                   data_1['label'].values,
                                                   test_size=0.33, random_state=7) 


In [130]:
vectorizer.vocabulary_

{'al jazeera': 0,
 'al qaeda': 1,
 'around world': 2,
 'bbc news': 3,
 'bin laden': 4,
 'climate change': 5,
 'david cameron': 6,
 'edward snowden': 7,
 'european union': 8,
 'first time': 9,
 'foreign minister': 10,
 'hong kong': 11,
 'human rights': 12,
 'islamic state': 13,
 'jong un': 14,
 'julian assange': 15,
 'kim jong': 16,
 'last year': 17,
 'middle east': 18,
 'new zealand': 19,
 'north korea': 20,
 'north korean': 21,
 'pirate bay': 22,
 'pope francis': 23,
 'prime minister': 24,
 'saudi arabia': 25,
 'security forces': 26,
 'shot dead': 27,
 'south africa': 28,
 'south korea': 29,
 'supreme court': 30,
 'united nations': 31,
 'united states': 32,
 'vladimir putin': 33,
 'war crimes': 34,
 'west bank': 35,
 'world cup': 36,
 'world war': 37,
 'year old': 38,
 'years ago': 39}

the vocabulary seems quite unspecific, probably this will be the reason for a bad performance

In [131]:
from sklearn.ensemble import RandomForestClassifier

In [132]:
#random forest model, parameters chosen from the previous experience with the algorithm
forest2 = RandomForestClassifier(n_estimators=100, max_depth=5, criterion='entropy', random_state=7,
                           n_jobs=-1, verbose=1)

forest2.fit(train_x, train_y)

[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False, random_state=7,
            verbose=1, warm_start=False)

In [133]:
forest2.score(train_x, train_y)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished


0.51316786374518897

In [134]:
forest2.score(test_x,test_y)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


0.5104288439057405

and again, chance level performance, although the predictions seem to be multi-class indeed (not that e.g. the algorithm always predicts a '0')

In [135]:
len(test_y[test_y == 0])/len(test_y)

0.5128284009106011

In [77]:
test_y

array([ 0.,  1.,  1., ...,  0.,  1.,  1.])

# Conclusion: possible reasons for a failure:
        1. Unsuitable dataset- maybe the top-25 Reddit news are not the valid source of information for DJIA prediction.
        There is a lack of such datasets, so one probably needs to use own data gathering methods like web scrapping 
        
        2. The news may just not have correlation with tomorrows DJIA change in principle
        
        3. Methods to convert the news into features did not extract meaningful information